In [2]:
import mysql.connector
import pandas as pd

# Connection with the database
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="rootpassword",
  database="recetas_db"
)

# Load data from the database into dataframes
ingredients_df = pd.read_sql_query("SELECT * FROM ingredientes", conn)
recipes_ingredients_df = pd.read_sql_query("SELECT * FROM recetas_ingredientes", conn)
allergens_df = pd.read_sql_query("SELECT * FROM alergenos", conn)
restriccions_df = pd.read_sql_query("SELECT * FROM restricciones", conn)

# Load data from the database into dataframes
conn.close()

C:\Users\jcela\AppData\Local\Temp\ipykernel_19428\2738617342.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ingredients_df = pd.read_sql_query("SELECT * FROM ingredientes", conn)
C:\Users\jcela\AppData\Local\Temp\ipykernel_19428\2738617342.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  recipes_ingredients_df = pd.read_sql_query("SELECT * FROM recetas_ingredientes", conn)
C:\Users\jcela\AppData\Local\Temp\ipykernel_19428\2738617342.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  allergens_df = pd.read_sql_qu

# First version, only 1 recipe

In [4]:
import pandas as pd
import numpy as np
import random
from typing import List
from dataclasses import dataclass
import mysql.connector

@dataclass
class Ingredient:
    id: int
    quantity: float
    total_fat: float 
    saturated_fats: float
    trans_fat: float
    cholesterol: float
    sodium: float
    carbohydrates: float
    protein: float
    calories: float

@dataclass
class Ant:
    recipe: List[Ingredient]

class AntColony:
    def __init__(self,
                 ingredients_df: pd.DataFrame,
                 recipes_ingredients_df: pd.DataFrame,
                 allergens_df: pd.DataFrame,
                 restriccions_df: pd.DataFrame,
                 hard_constraints: str,
                 num_ants: int = 10, 
                 evaporation_rate: float = 0.5,
                 alpha: int = 1,
                 beta: int = 2,
                 pheromone_init: float = 0.1,
                 pheromone_deposit: int = 1,
                 max_iterations: int = 100) -> None:
        self.ingredients_df = ingredients_df
        self.recipes_ingredients_df = recipes_ingredients_df
        self.allergens_df = allergens_df
        self.restriccions_df = restriccions_df
        self.hard_constraints = hard_constraints
        self.num_ants = num_ants
        self.evaporation_rate = evaporation_rate
        self.alpha = alpha
        self.beta = beta
        self.pheromone_init = pheromone_init
        self.pheromone_deposit = pheromone_deposit
        self.max_iterations = max_iterations
        self.num_ingredients = len(self.ingredients_df)
        self.pheromone_matrix = np.full((self.num_ingredients, self.num_ingredients), pheromone_init)
        self.best_solution = None
        self.best_solution_fitness = float('inf')

    def _optimize(self) -> List[int]:
        for _ in range(self.max_iterations):
            solutions = self._construct_solution()
            for solution in solutions:
                solution_fitness = self.evaluate_solution_fitness(solution)
                if solution_fitness < self.best_solution_fitness:
                    self.best_solution = solution
                    self.best_solution_fitness = solution_fitness
            self.update_pheromone_matrix(solutions)
        return self.best_solution
    
    def _construct_solution(self) -> List[Ant]:
        ants = [Ant(recipe=[]) for _ in range(self.num_ants)]
        for ant in ants:
            recipe_length = random.randint(5, 15)
            selected_ingredients = set() # to avoid repeating ingredients
            while len(ant.recipe) < recipe_length:
                next_ingredient = self._select_next_ingredient(ant.recipe, selected_ingredients)
                if next_ingredient is not None:
                    ant.recipe.append(next_ingredient)
                    selected_ingredients.add(next_ingredient.id)
                else:
                    break  # No more valid ingredients
        return ants
    
    def _select_next_ingredient(self, current_recipe: List[Ingredient], selected_ingredients: set) -> Ingredient:
        probabilities = []
        available_ingredients = [i for i in range(1, self.num_ingredients + 1) if i not in selected_ingredients]
        
        for ingredient_id in available_ingredients:
            if self._check_ingredient(ingredient_id):
                #print(f'{ingredient_id} IS VALID!')
                pheromone = sum(self.pheromone_matrix[ingredient_id - 1][i.id - 1] for i in current_recipe) if current_recipe else 1.0
                desirability = self._calculate_desirability(ingredient_id)
                probability = (pheromone ** self.alpha) * (desirability ** self.beta)
                #print(f'Ingredient: {ingredient_id}, Pheromone: {pheromone}, Desirability: {desirability}, Probability: {probability}')
                probabilities.append((ingredient_id, probability))
        
        if not probabilities:
            #print("No valid ingredients found.")
            return None
        
        total_probability = sum(prob for _, prob in probabilities)
        if total_probability == 0:
            #print("Total probability is zero.")
            return None

        # Normalización de las probabilidades
        probabilities = [(ing, prob / total_probability) for ing, prob in probabilities]
        #print(f'Probabilities: {probabilities}')
        
        selected_ingredient_id = random.choices(
            [ing for ing, _ in probabilities], 
            [prob for _, prob in probabilities]
        )[0]
        
        selected_ingredient = self._create_ingredient(selected_ingredient_id)
        #print(f'WILL ADD: {selected_ingredient}')
        return selected_ingredient

    
    def _create_ingredient(self, ingredient_id: int) -> Ingredient:
        ingredient_data = self.recipes_ingredients_df[self.recipes_ingredients_df['ID_INGREDIENTE'] == ingredient_id].sample()
        ingredient_row = ingredient_data.iloc[0]
        return Ingredient(
            id=ingredient_id,
            quantity=ingredient_row['Cantidad'],
            total_fat=ingredient_row['Grasa'],
            saturated_fats=ingredient_row['Grasas_saturadas'],
            trans_fat=ingredient_row['Grasas_trans'],
            cholesterol=ingredient_row['Colesterol'],
            sodium=ingredient_row['Sodio'],
            carbohydrates=ingredient_row['Carbohidratos'],
            protein=ingredient_row['Proteina'],
            calories=ingredient_row['Calorias']
        )

    def _calculate_desirability(self, ingredient_id: int) -> float:
        return 1.0

    def _check_ingredient(self, ingredient_id: int) -> bool:
        constraints_list = self.hard_constraints.split(',')
        vegan = int(constraints_list[0])
        vegetarian = int(constraints_list[1])
        user_allergens = [int(x) for x in constraints_list[2:]]
        
        ingredient = self.ingredients_df[self.ingredients_df['ID'] == ingredient_id]
        if ingredient.empty:
            raise ValueError(f'Ingredient with id: {ingredient_id} not found')
        
        ingredient_vegan = ingredient['Vegano'].iloc[0]
        ingredient_vegetarian = ingredient['Vegetariano'].iloc[0]
        
        if vegan and not ingredient_vegan:
            return False
        if vegetarian and not ingredient_vegetarian:
            return False
        
        ingredient_allergens = self.restriccions_df[self.restriccions_df['ID_INGREDIENTE'] == ingredient_id]['ID_ALERGENO'].tolist()
        if set(user_allergens).intersection(set(ingredient_allergens)):
            return False
        
        return True
    
    def evaluate_solution_fitness(self, solution: Ant) -> float:
        total_nutrition = self._calculate_total_nutrition(solution.recipe)
        fitness = 0

        # Define the target ranges
        target_ranges = {
            'Grasa': (13, 27),
            'Grasas_saturadas': (5, 7),
            'Grasas_trans': (0, 0.3),
            'Colesterol': (0.05, 0.1),
            'Sodio': (0.5, 0.8),
            'Carbohidratos': (75, 108),
            'Proteina': (15, 35),
            'Calorias': (600, 700)
        }

        # Calculate the fitness score
        for nutrient, (min_val, max_val) in target_ranges.items():
            if total_nutrition[nutrient] < min_val:
                fitness += (min_val - total_nutrition[nutrient]) ** 2
            elif total_nutrition[nutrient] > max_val:
                fitness += (total_nutrition[nutrient] - max_val) ** 2

        return fitness
    
    def _calculate_total_nutrition(self, recipe: List[Ingredient]) -> pd.Series:
        total_nutrition = pd.Series(
            data={
                'Cantidad': 0,
                'Grasa': 0,
                'Grasas_saturadas': 0,
                'Grasas_trans': 0,
                'Colesterol': 0,
                'Sodio': 0,
                'Carbohidratos': 0,
                'Proteina': 0,
                'Calorias': 0
            }
        )
        for ingredient in recipe:
            total_nutrition += pd.Series(
                data={
                    'Cantidad': ingredient.quantity,
                    'Grasa': ingredient.total_fat,
                    'Grasas_saturadas': ingredient.saturated_fats,
                    'Grasas_trans': ingredient.trans_fat,
                    'Colesterol': ingredient.cholesterol,
                    'Sodio': ingredient.sodium,
                    'Carbohidratos': ingredient.carbohydrates,
                    'Proteina': ingredient.protein,
                    'Calorias': ingredient.calories
                }
            )
        return total_nutrition
    
    def update_pheromone_matrix(self, solutions: List[Ant]) -> None:
        self.pheromone_matrix *= (1 - self.evaporation_rate)
        for ant in solutions:
            fitness = self.evaluate_solution_fitness(ant)
            if fitness == 0:
                fitness = 1  # Avoid division by zero
            pheromone_contribution = self.pheromone_deposit / fitness
            for i in range(len(ant.recipe) - 1):
                self.pheromone_matrix[ant.recipe[i].id -1][ant.recipe[i+1].id - 1] += pheromone_contribution

if __name__ == "__main__":
    # First number: Vegan (0 no, 1 yes)
    # Second number: Vegetarian (0 no, 1 yes)
    # Rest of numbers: Ids of alegerns
    hard_constraints = '0,0,1'
    ant_colony = AntColony(
        ingredients_df=ingredients_df, 
        recipes_ingredients_df=recipes_ingredients_df, 
        allergens_df=allergens_df, 
        restriccions_df=restriccions_df, 
        hard_constraints=hard_constraints
    )
    best_recipe = ant_colony._optimize()
    if best_recipe:
        recipe_ingredients = pd.DataFrame([ingredient.__dict__ for ingredient in best_recipe.recipe])
        merged_df = recipe_ingredients.merge(ingredients_df, left_on='id', right_on='ID')
        for index, row in merged_df.iterrows():
            print(f'''ID: {row['id']},
                  Name: {row['Nombre']},
                  Quantity: {row['quantity']}, 
                  Total Fat: {row['total_fat']}, Saturated Fats: {row['saturated_fats']},
                  Trans Fat: {row['trans_fat']}, Cholesterol: {row['cholesterol']},
                  Sodium: {row['sodium']}, Carbohydrates: {row['carbohydrates']},
                  Protein: {row['protein']}, Calories: {row['calories']}''')
    else:
        print("No valid recipe found.")

ID: 49,
                  Name: Sobre especias italianas,
                  Quantity: 2.7, 
                  Total Fat: 0.2, Saturated Fats: 0.1,
                  Trans Fat: 0.0, Cholesterol: 0.0,
                  Sodium: 0.0, Carbohydrates: 1.7,
                  Protein: 0.2, Calories: 7.5
ID: 2,
                  Name: Jengibre,
                  Quantity: 45.0, 
                  Total Fat: 0.3, Saturated Fats: 0.1,
                  Trans Fat: 0.0, Cholesterol: 0.0,
                  Sodium: 0.06, Carbohydrates: 8.0,
                  Protein: 0.8, Calories: 36.0
ID: 21,
                  Name: Tomate triturado,
                  Quantity: 400.0, 
                  Total Fat: 1.1, Saturated Fats: 0.2,
                  Trans Fat: 0.0, Cholesterol: 0.0,
                  Sodium: 0.74, Carbohydrates: 29.2,
                  Protein: 6.6, Calories: 128.0
ID: 3,
                  Name: Cilantro,
                  Quantity: 50.0, 
                  Total Fat: 0.3, Saturated Fats: 0.

# Second version, 7 recipes

In [16]:
import pandas as pd
import numpy as np
import random
from typing import List
from dataclasses import dataclass
import mysql.connector

@dataclass
class Ingredient:
    id: int
    quantity: float
    total_fat: float 
    saturated_fats: float
    trans_fat: float
    cholesterol: float
    sodium: float
    carbohydrates: float
    protein: float
    calories: float

@dataclass
class Ant:
    recipe: List[Ingredient]

class AntColony:
    def __init__(self,
                 ingredients_df: pd.DataFrame,
                 recipes_ingredients_df: pd.DataFrame,
                 allergens_df: pd.DataFrame,
                 restriccions_df: pd.DataFrame,
                 hard_constraints: str,
                 num_ants: int = 10, 
                 evaporation_rate: float = 0.5,
                 alpha: int = 1,
                 beta: int = 2,
                 pheromone_init: float = 0.1,
                 pheromone_deposit: int = 1,
                 max_iterations: int = 100) -> None:
        self.ingredients_df = ingredients_df
        self.recipes_ingredients_df = recipes_ingredients_df
        self.allergens_df = allergens_df
        self.restriccions_df = restriccions_df
        self.hard_constraints = hard_constraints
        self.num_ants = num_ants
        self.evaporation_rate = evaporation_rate
        self.alpha = alpha
        self.beta = beta
        self.pheromone_init = pheromone_init
        self.pheromone_deposit = pheromone_deposit
        self.max_iterations = max_iterations
        self.num_ingredients = len(self.ingredients_df)
        self.pheromone_matrix = np.full((self.num_ingredients, self.num_ingredients), pheromone_init)
        self.best_solutions = []
        self.num_recipes = 7  # Número de recetas a generar

    def _optimize(self) -> List[List[Ingredient]]:
        for _ in range(self.max_iterations):
            solutions = self._construct_solution()
            for solution in solutions:
                solution_fitness = self.evaluate_solution_fitness(solution)
                if len(self.best_solutions) < self.num_recipes:
                    self.best_solutions.append((solution, solution_fitness))
                else:
                    worst_fitness = max(self.best_solutions, key=lambda x: x[1])[1]
                    if solution_fitness < worst_fitness:
                        self.best_solutions.remove(max(self.best_solutions, key=lambda x: x[1]))
                        self.best_solutions.append((solution, solution_fitness))
            self.update_pheromone_matrix(solutions)
        return [solution[0].recipe for solution in self.best_solutions]
    
    def _construct_solution(self) -> List[Ant]:
        ants = [Ant(recipe=[]) for _ in range(self.num_ants)]
        for ant in ants:
            recipe_length = random.randint(5, 15)
            selected_ingredients = set()  # Para evitar repetir ingredientes
            while len(ant.recipe) < recipe_length:
                next_ingredient = self._select_next_ingredient(ant.recipe, selected_ingredients)
                if next_ingredient is not None:
                    ant.recipe.append(next_ingredient)
                    selected_ingredients.add(next_ingredient.id)
                else:
                    break  # No hay más ingredientes válidos
        return ants
    
    def _select_next_ingredient(self, current_recipe: List[Ingredient], selected_ingredients: set) -> Ingredient:
        probabilities = []
        available_ingredients = [i for i in range(1, self.num_ingredients + 1) if i not in selected_ingredients]
        
        for ingredient_id in available_ingredients:
            if self._check_ingredient(ingredient_id):
                pheromone = sum(self.pheromone_matrix[ingredient_id - 1][i.id - 1] for i in current_recipe) if current_recipe else 1.0
                desirability = self._calculate_desirability(ingredient_id)
                probability = (pheromone ** self.alpha) * (desirability ** self.beta)
                probabilities.append((ingredient_id, probability))
        
        if not probabilities:
            return None
        
        total_probability = sum(prob for _, prob in probabilities)
        if total_probability == 0:
            return None

        probabilities = [(ing, prob / total_probability) for ing, prob in probabilities]
        
        selected_ingredient_id = random.choices(
            [ing for ing, _ in probabilities], 
            [prob for _, prob in probabilities]
        )[0]
        
        return self._create_ingredient(selected_ingredient_id)

    def _create_ingredient(self, ingredient_id: int) -> Ingredient:
        ingredient_data = self.recipes_ingredients_df[self.recipes_ingredients_df['ID_INGREDIENTE'] == ingredient_id].sample()
        ingredient_row = ingredient_data.iloc[0]
        return Ingredient(
            id=ingredient_id,
            quantity=ingredient_row['Cantidad'],
            total_fat=ingredient_row['Grasa'],
            saturated_fats=ingredient_row['Grasas_saturadas'],
            trans_fat=ingredient_row['Grasas_trans'],
            cholesterol=ingredient_row['Colesterol'],
            sodium=ingredient_row['Sodio'],
            carbohydrates=ingredient_row['Carbohidratos'],
            protein=ingredient_row['Proteina'],
            calories=ingredient_row['Calorias']
        )

    def _calculate_desirability(self, ingredient_id: int) -> float:
        return 1.0

    def _check_ingredient(self, ingredient_id: int) -> bool:
        constraints_list = self.hard_constraints.split(',')
        vegan = int(constraints_list[0])
        vegetarian = int(constraints_list[1])
        user_allergens = [int(x) for x in constraints_list[2:]]
        
        ingredient = self.ingredients_df[self.ingredients_df['ID'] == ingredient_id]
        if ingredient.empty:
            raise ValueError(f'Ingredient with id: {ingredient_id} not found')
        
        ingredient_vegan = ingredient['Vegano'].iloc[0]
        ingredient_vegetarian = ingredient['Vegetariano'].iloc[0]
        
        if vegan and not ingredient_vegan:
            return False
        if vegetarian and not ingredient_vegetarian:
            return False
        
        ingredient_allergens = self.restriccions_df[self.restriccions_df['ID_INGREDIENTE'] == ingredient_id]['ID_ALERGENO'].tolist()
        if set(user_allergens).intersection(set(ingredient_allergens)):
            return False
        
        return True
    
    def evaluate_solution_fitness(self, solution: Ant) -> float:
        total_nutrition = self._calculate_total_nutrition(solution.recipe)
        fitness = 0

        target_ranges = {
            'Grasa': (13, 27),
            'Grasas_saturadas': (5, 7),
            'Grasas_trans': (0, 0.3),
            'Colesterol': (0.05, 0.1),
            'Sodio': (0.5, 0.8),
            'Carbohidratos': (75, 108),
            'Proteina': (15, 35),
            'Calorias': (600, 700)
        }

        for nutrient, (min_val, max_val) in target_ranges.items():
            if total_nutrition[nutrient] < min_val:
                fitness += (min_val - total_nutrition[nutrient]) ** 2
            elif total_nutrition[nutrient] > max_val:
                fitness += (total_nutrition[nutrient] - max_val) ** 2

        return fitness
    
    def _calculate_total_nutrition(self, recipe: List[Ingredient]) -> pd.Series:
        total_nutrition = pd.Series(
            data={
                'Cantidad': 0,
                'Grasa': 0,
                'Grasas_saturadas': 0,
                'Grasas_trans': 0,
                'Colesterol': 0,
                'Sodio': 0,
                'Carbohidratos': 0,
                'Proteina': 0,
                'Calorias': 0
            }
        )
        for ingredient in recipe:
            total_nutrition += pd.Series(
                data={
                    'Cantidad': ingredient.quantity,
                    'Grasa': ingredient.total_fat,
                    'Grasas_saturadas': ingredient.saturated_fats,
                    'Grasas_trans': ingredient.trans_fat,
                    'Colesterol': ingredient.cholesterol,
                    'Sodio': ingredient.sodium,
                    'Carbohidratos': ingredient.carbohydrates,
                    'Proteina': ingredient.protein,
                    'Calorias': ingredient.calories
                }
            )
        return total_nutrition
    
    def update_pheromone_matrix(self, solutions: List[Ant]) -> None:
        self.pheromone_matrix *= (1 - self.evaporation_rate)
        for ant in solutions:
            fitness = self.evaluate_solution_fitness(ant)
            if fitness == 0:
                fitness = 1  # Evitar división por cero
            pheromone_contribution = self.pheromone_deposit / fitness
            for i in range(len(ant.recipe) - 1):
                 self.pheromone_matrix[ant.recipe[i].id - 1][ant.recipe[i+1].id - 1] += pheromone_contribution

if __name__ == "__main__":
    # First number: Vegan (0 no, 1 yes)
    # Second number: Vegetarian (0 no, 1 yes)
    # Rest of numbers: Ids of allergens
    hard_constraints = '0,0,1'
    
    # Suponiendo que ya tienes tus DataFrames cargados:
    # ingredients_df, recipes_ingredients_df, allergens_df, restriccions_df

    ant_colony = AntColony(
        ingredients_df=ingredients_df, 
        recipes_ingredients_df=recipes_ingredients_df, 
        allergens_df=allergens_df, 
        restriccions_df=restriccions_df, 
        hard_constraints=hard_constraints
    )
    
    best_recipes = ant_colony._optimize()
    
    if best_recipes:
        for i, best_recipe in enumerate(best_recipes):
            print(f"Receta {i+1}:")
            recipe_ingredients = pd.DataFrame([ingredient.__dict__ for ingredient in best_recipe])
            merged_df = recipe_ingredients.merge(ingredients_df, left_on='id', right_on='ID')
            for index, row in merged_df.iterrows():
                print(f'''ID: {row['id']},
                      Name: {row['Nombre']},
                      Quantity: {row['quantity']}, 
                      Total Fat: {row['total_fat']}, Saturated Fats: {row['saturated_fats']},
                      Trans Fat: {row['trans_fat']}, Cholesterol: {row['cholesterol']},
                      Sodium: {row['sodium']}, Carbohydrates: {row['carbohydrates']},
                      Protein: {row['protein']}, Calories: {row['calories']}''')
            print("\n")
    else:
        print("No valid recipes found.")

Receta 1:
ID: 41,
                      Name: Menta,
                      Quantity: 100.0, 
                      Total Fat: 0.7, Saturated Fats: 0.2,
                      Trans Fat: 0.0, Cholesterol: 0.0,
                      Sodium: 0.03, Carbohydrates: 8.4,
                      Protein: 3.3, Calories: 44.0
ID: 51,
                      Name: Vinagre balsÃ¡mico,
                      Quantity: 5.0, 
                      Total Fat: 0.0, Saturated Fats: 0.0,
                      Trans Fat: 0.0, Cholesterol: 0.0,
                      Sodium: 0.0, Carbohydrates: 0.9,
                      Protein: 0.0, Calories: 4.4
ID: 71,
                      Name: Brotes de espinacas,
                      Quantity: 284.0, 
                      Total Fat: 1.1, Saturated Fats: 0.2,
                      Trans Fat: 0.0, Cholesterol: 0.0,
                      Sodium: 0.22, Carbohydrates: 10.3,
                      Protein: 8.1, Calories: 65.4
ID: 54,
                      Name: Mostaza,
      

## Statistical tests

In [17]:
import time
import statistics as stats
from scipy.stats import ttest_1samp, wilcoxon
import numpy as np

def run_experiment(ant_colony, num_runs):
    fitness_scores = []
    convergence_rates = []
    computational_times = []

    for _ in range(num_runs):
        start_time = time.time()
        best_recipes = ant_colony._optimize()
        end_time = time.time()

        # Evaluate fitness scores
        run_fitness_scores = [ant_colony.evaluate_solution_fitness(Ant(recipe=recipe)) for recipe in best_recipes]
        fitness_scores.extend(run_fitness_scores)

        # Calculate convergence rate
        unique_fitness_scores = set(run_fitness_scores)
        convergence_rate = len(unique_fitness_scores) / len(run_fitness_scores)
        convergence_rates.append(convergence_rate)

        # Record computational time
        computational_times.append(end_time - start_time)

    return {
        "fitness_scores": fitness_scores,
        "average_fitness_score": stats.mean(fitness_scores),
        "convergence_rate": stats.mean(convergence_rates),
        "computational_time": stats.mean(computational_times),
        "fitness_score_std_dev": stats.stdev(fitness_scores)
    }

# Run the experiment
num_runs = 5
experiment_results = run_experiment(ant_colony, num_runs)

# Output results
print("Average Fitness Score:", experiment_results["average_fitness_score"])
print("Convergence Rate:", experiment_results["convergence_rate"])
print("Computational Time:", experiment_results["computational_time"])
print("Fitness Score Standard Deviation:", experiment_results["fitness_score_std_dev"])

# Additional tests
fitness_scores_array = np.array(experiment_results["fitness_scores"])

# Mean and Standard Deviation
print("Mean of Fitness Scores:", np.mean(fitness_scores_array))
print("Standard Deviation of Fitness Scores:", np.std(fitness_scores_array))

# T-tests or ANOVA
# Example t-test comparing to a hypothetical mean value (e.g., 100)
t_statistic, p_value = ttest_1samp(fitness_scores_array, 100)
print("T-statistic:", t_statistic)
print("P-value:", p_value)

# Wilcoxon Signed-Rank Test
# Example Wilcoxon signed-rank test comparing to a hypothetical median value (e.g., 100)
w_statistic, p_value = wilcoxon(fitness_scores_array - 100)
print("Wilcoxon Signed-Rank Test Statistic:", w_statistic)
print("P-value:", p_value)


Average Fitness Score: 0.008291428571428568
Convergence Rate: 0.5714285714285714
Computational Time: 199.1956494808197
Fitness Score Standard Deviation: 0.009173984041477143
Mean of Fitness Scores: 0.008291428571428568
Standard Deviation of Fitness Scores: 0.009041977388936603
T-statistic: -64482.227446928046
P-value: 4.438009150094571e-139
Wilcoxon Signed-Rank Test Statistic: 0.0
P-value: 5.820766091346741e-11


# Third version, 7 recipes and metrics to compare similarity between existing recipes.

## Jaccard distance

In [14]:
import pandas as pd
import numpy as np
import random
from typing import List
from dataclasses import dataclass
from scipy.spatial.distance import jaccard

@dataclass
class Ingredient:
    id: int
    quantity: float
    total_fat: float 
    saturated_fats: float
    trans_fat: float
    cholesterol: float
    sodium: float
    carbohydrates: float
    protein: float
    calories: float

@dataclass
class Ant:
    recipe: List[Ingredient]

class AntColony:
    def __init__(self,
                 ingredients_df: pd.DataFrame,
                 recipes_ingredients_df: pd.DataFrame,
                 allergens_df: pd.DataFrame,
                 restriccions_df: pd.DataFrame,
                 hard_constraints: str,
                 num_ants: int = 10, 
                 evaporation_rate: float = 0.5,
                 alpha: int = 1,
                 beta: int = 2,
                 pheromone_init: float = 0.1,
                 pheromone_deposit: int = 1,
                 max_iterations: int = 100) -> None:
        self.ingredients_df = ingredients_df
        self.recipes_ingredients_df = recipes_ingredients_df
        self.allergens_df = allergens_df
        self.restriccions_df = restriccions_df
        self.hard_constraints = hard_constraints
        self.num_ants = num_ants
        self.evaporation_rate = evaporation_rate
        self.alpha = alpha
        self.beta = beta
        self.pheromone_init = pheromone_init
        self.pheromone_deposit = pheromone_deposit
        self.max_iterations = max_iterations
        self.num_ingredients = len(self.ingredients_df)
        self.pheromone_matrix = np.full((self.num_ingredients, self.num_ingredients), pheromone_init)
        self.best_solutions = []
        self.num_recipes = 7  # Number of recipes to generate

    def _optimize(self) -> List[List[Ingredient]]:
        for _ in range(self.max_iterations):
            solutions = self._construct_solution()
            for solution in solutions:
                solution_fitness = self.evaluate_solution_fitness(solution)
                if len(self.best_solutions) < self.num_recipes:
                    self.best_solutions.append((solution, solution_fitness))
                else:
                    worst_fitness = max(self.best_solutions, key=lambda x: x[1])[1]
                    if solution_fitness < worst_fitness:
                        self.best_solutions.remove(max(self.best_solutions, key=lambda x: x[1]))
                        self.best_solutions.append((solution, solution_fitness))
            
            # Compare the generated recipe with existing ones and replace if necessary
            for i in range(len(solutions)):
                for j in range(len(self.best_solutions)):
                    generated_recipe = [ing.id for ing in solutions[i].recipe]
                    existing_recipe = [ing.id for ing in self.best_solutions[j][0].recipe]
                    max_length = max(len(generated_recipe), len(existing_recipe))
                    generated_recipe += [0] * (max_length - len(generated_recipe))
                    existing_recipe += [0] * (max_length - len(existing_recipe))
                    jaccard_distance = jaccard(generated_recipe, existing_recipe)
                    if jaccard_distance < 0.5:  # Jaccard distance threshold
                        self.best_solutions[j] = (solutions[i], self.evaluate_solution_fitness(solutions[i]))
            
            self.update_pheromone_matrix(solutions)
        return [solution[0].recipe for solution in self.best_solutions]
    
    def _construct_solution(self) -> List[Ant]:
        ants = [Ant(recipe=[]) for _ in range(self.num_ants)]
        for ant in ants:
            recipe_length = random.randint(5, 15)
            selected_ingredients = set()  # Para evitar repetir ingredientes
            while len(ant.recipe) < recipe_length:
                next_ingredient = self._select_next_ingredient(ant.recipe, selected_ingredients)
                if next_ingredient is not None:
                    ant.recipe.append(next_ingredient)
                    selected_ingredients.add(next_ingredient.id)
                else:
                    break  # No hay más ingredientes válidos
        return ants
    
    def _select_next_ingredient(self, current_recipe: List[Ingredient], selected_ingredients: set) -> Ingredient:
        probabilities = []
        available_ingredients = [i for i in range(1, self.num_ingredients + 1) if i not in selected_ingredients]
        
        for ingredient_id in available_ingredients:
            if self._check_ingredient(ingredient_id):
                pheromone = sum(self.pheromone_matrix[ingredient_id - 1][i.id - 1] for i in current_recipe) if current_recipe else 1.0
                desirability = self._calculate_desirability(ingredient_id)
                probability = (pheromone ** self.alpha) * (desirability ** self.beta)
                probabilities.append((ingredient_id, probability))
        
        if not probabilities:
            return None
        
        total_probability = sum(prob for _, prob in probabilities)
        if total_probability == 0:
            return None

        probabilities = [(ing, prob / total_probability) for ing, prob in probabilities]
        
        selected_ingredient_id = random.choices(
            [ing for ing, _ in probabilities], 
            [prob for _, prob in probabilities]
        )[0]
        
        return self._create_ingredient(selected_ingredient_id)

    def _create_ingredient(self, ingredient_id: int) -> Ingredient:
        ingredient_data = self.recipes_ingredients_df[self.recipes_ingredients_df['ID_INGREDIENTE'] == ingredient_id].sample()
        ingredient_row = ingredient_data.iloc[0]
        return Ingredient(
            id=ingredient_id,
            quantity=ingredient_row['Cantidad'],
            total_fat=ingredient_row['Grasa'],
            saturated_fats=ingredient_row['Grasas_saturadas'],
            trans_fat=ingredient_row['Grasas_trans'],
            cholesterol=ingredient_row['Colesterol'],
            sodium=ingredient_row['Sodio'],
            carbohydrates=ingredient_row['Carbohidratos'],
            protein=ingredient_row['Proteina'],
            calories=ingredient_row['Calorias']
        )

    def _calculate_desirability(self, ingredient_id: int) -> float:
        return 1.0

    def _check_ingredient(self, ingredient_id: int) -> bool:
        constraints_list = self.hard_constraints.split(',')
        vegan = int(constraints_list[0])
        vegetarian = int(constraints_list[1])
        user_allergens = [int(x) for x in constraints_list[2:]]
        
        ingredient = self.ingredients_df[self.ingredients_df['ID'] == ingredient_id]
        if ingredient.empty:
            raise ValueError(f'Ingredient with id: {ingredient_id} not found')
        
        ingredient_vegan = ingredient['Vegano'].iloc[0]
        ingredient_vegetarian = ingredient['Vegetariano'].iloc[0]
        
        if vegan and not ingredient_vegan:
            return False
        if vegetarian and not ingredient_vegetarian:
            return False
        
        ingredient_allergens = self.restriccions_df[self.restriccions_df['ID_INGREDIENTE'] == ingredient_id]['ID_ALERGENO'].tolist()
        if set(user_allergens).intersection(set(ingredient_allergens)):
            return False
        
        return True
    
    def evaluate_solution_fitness(self, solution: Ant) -> float:
        total_nutrition = self._calculate_total_nutrition(solution.recipe)
        fitness = 0

        target_ranges = {
            'Grasa': (13, 27),
            'Grasas_saturadas': (5, 7),
            'Grasas_trans': (0, 0.3),
            'Colesterol': (0.05, 0.1),
            'Sodio': (0.5, 0.8),
            'Carbohidratos': (75, 108),
            'Proteina': (15, 35),
            'Calorias': (600, 700)
        }

        for nutrient, (min_val, max_val) in target_ranges.items():
            if total_nutrition[nutrient] < min_val:
                fitness += (min_val - total_nutrition[nutrient]) ** 2
            elif total_nutrition[nutrient] > max_val:
                fitness += (total_nutrition[nutrient] - max_val) ** 2

        return fitness
    
    def _calculate_total_nutrition(self, recipe: List[Ingredient]) -> pd.Series:
        total_nutrition = pd.Series(
            data={
                'Cantidad': 0,
                'Grasa': 0,
                'Grasas_saturadas': 0,
                'Grasas_trans': 0,
                'Colesterol': 0,
                'Sodio': 0,
                'Carbohidratos': 0,
                'Proteina': 0,
                'Calorias': 0
            }
        )
        for ingredient in recipe:
            total_nutrition += pd.Series(
                data={
                    'Cantidad': ingredient.quantity,
                    'Grasa': ingredient.total_fat,
                    'Grasas_saturadas': ingredient.saturated_fats,
                    'Grasas_trans': ingredient.trans_fat,
                    'Colesterol': ingredient.cholesterol,
                    'Sodio': ingredient.sodium,
                    'Carbohidratos': ingredient.carbohydrates,
                    'Proteina': ingredient.protein,
                    'Calorias': ingredient.calories
                }
            )
        return total_nutrition
    
    def update_pheromone_matrix(self, solutions: List[Ant]) -> None:
        self.pheromone_matrix *= (1 - self.evaporation_rate)
        for ant in solutions:
            fitness = self.evaluate_solution_fitness(ant)
            if fitness == 0:
                fitness = 1  # Evitar división por cero
            pheromone_contribution = self.pheromone_deposit / fitness
            for i in range(len(ant.recipe) - 1):
                 self.pheromone_matrix[ant.recipe[i].id - 1][ant.recipe[i+1].id - 1] += pheromone_contribution

if __name__ == "__main__":
    # First number: Vegan (0 no, 1 yes)
    # Second number: Vegetarian (0 no, 1 yes)
    # Rest of numbers: Ids of allergens
    hard_constraints = '0,0,1'

    ant_colony = AntColony(
        ingredients_df=ingredients_df, 
        recipes_ingredients_df=recipes_ingredients_df, 
        allergens_df=allergens_df, 
        restriccions_df=restriccions_df, 
        hard_constraints=hard_constraints
    )
    
    best_recipes = ant_colony._optimize()
    
    if best_recipes:
        for i, best_recipe in enumerate(best_recipes):
            print(f"Receta {i+1}:")
            recipe_ingredients = pd.DataFrame([ingredient.__dict__ for ingredient in best_recipe])
            merged_df = recipe_ingredients.merge(ingredients_df, left_on='id', right_on='ID')
            for index, row in merged_df.iterrows():
                print(f'''ID: {row['id']},
                      Name: {row['Nombre']},
                      Quantity: {row['quantity']}, 
                      Total Fat: {row['total_fat']}, Saturated Fats: {row['saturated_fats']},
                      Trans Fat: {row['trans_fat']}, Cholesterol: {row['cholesterol']},
                      Sodium: {row['sodium']}, Carbohydrates: {row['carbohydrates']},
                      Protein: {row['protein']}, Calories: {row['calories']}''')
            print("\n")
    else:
        print("No valid recipes found.")

Receta 1:
ID: 47,
                      Name: Tomate frito,
                      Quantity: 200.0, 
                      Total Fat: 0.4, Saturated Fats: 0.1,
                      Trans Fat: 0.0, Cholesterol: 0.0,
                      Sodium: 0.01, Carbohydrates: 7.8,
                      Protein: 1.8, Calories: 36.0
ID: 35,
                      Name: Tomate,
                      Quantity: 123.0, 
                      Total Fat: 0.2, Saturated Fats: 0.0,
                      Trans Fat: 0.0, Cholesterol: 0.0,
                      Sodium: 0.01, Carbohydrates: 4.8,
                      Protein: 1.1, Calories: 22.1
ID: 2,
                      Name: Jengibre,
                      Quantity: 45.0, 
                      Total Fat: 0.3, Saturated Fats: 0.1,
                      Trans Fat: 0.0, Cholesterol: 0.0,
                      Sodium: 0.06, Carbohydrates: 8.0,
                      Protein: 0.8, Calories: 36.0
ID: 31,
                      Name: Puerro,
                      

## Statistical tests Jaccard distance

In [15]:
import time
import statistics as stats
from scipy.stats import ttest_1samp, wilcoxon
import numpy as np

def run_experiment(ant_colony, num_runs):
    fitness_scores = []
    convergence_rates = []
    computational_times = []

    for _ in range(num_runs):
        start_time = time.time()
        best_recipes = ant_colony._optimize()
        end_time = time.time()

        # Evaluate fitness scores
        run_fitness_scores = [ant_colony.evaluate_solution_fitness(Ant(recipe=recipe)) for recipe in best_recipes]
        fitness_scores.extend(run_fitness_scores)

        # Calculate convergence rate
        unique_fitness_scores = set(run_fitness_scores)
        convergence_rate = len(unique_fitness_scores) / len(run_fitness_scores)
        convergence_rates.append(convergence_rate)

        # Record computational time
        computational_times.append(end_time - start_time)

    return {
        "fitness_scores": fitness_scores,
        "average_fitness_score": stats.mean(fitness_scores),
        "convergence_rate": stats.mean(convergence_rates),
        "computational_time": stats.mean(computational_times),
        "fitness_score_std_dev": stats.stdev(fitness_scores)
    }

# Run the experiment
num_runs = 5
experiment_results = run_experiment(ant_colony, num_runs)

# Output results
print("Average Fitness Score:", experiment_results["average_fitness_score"])
print("Convergence Rate:", experiment_results["convergence_rate"])
print("Computational Time:", experiment_results["computational_time"])
print("Fitness Score Standard Deviation:", experiment_results["fitness_score_std_dev"])

# Additional tests
fitness_scores_array = np.array(experiment_results["fitness_scores"])

# Mean and Standard Deviation
print("Mean of Fitness Scores:", np.mean(fitness_scores_array))
print("Standard Deviation of Fitness Scores:", np.std(fitness_scores_array))

# T-tests or ANOVA
# Example t-test comparing to a hypothetical mean value (e.g., 100)
t_statistic, p_value = ttest_1samp(fitness_scores_array, 100)
print("T-statistic:", t_statistic)
print("P-value:", p_value)

# Wilcoxon Signed-Rank Test
# Example Wilcoxon signed-rank test comparing to a hypothetical median value (e.g., 100)
w_statistic, p_value = wilcoxon(fitness_scores_array - 100)
print("Wilcoxon Signed-Rank Test Statistic:", w_statistic)
print("P-value:", p_value)


Average Fitness Score: 4.759677142857162
Convergence Rate: 0.8857142857142857
Computational Time: 198.28439993858336
Fitness Score Standard Deviation: 8.024767795324342
Mean of Fitness Scores: 4.7596771428571625
Standard Deviation of Fitness Scores: 7.909297490461534
T-statistic: -70.213788481122
P-value: 2.190049862568523e-38
Wilcoxon Signed-Rank Test Statistic: 0.0
P-value: 5.820766091346741e-11


## Hamming distance

In [8]:
import pandas as pd
import numpy as np
import random
from typing import List
from dataclasses import dataclass

@dataclass
class Ingredient:
    id: int
    quantity: float
    total_fat: float 
    saturated_fats: float
    trans_fat: float
    cholesterol: float
    sodium: float
    carbohydrates: float
    protein: float
    calories: float

@dataclass
class Ant:
    recipe: List[Ingredient]

class AntColony:
    def __init__(self,
                 ingredients_df: pd.DataFrame,
                 recipes_ingredients_df: pd.DataFrame,
                 allergens_df: pd.DataFrame,
                 restriccions_df: pd.DataFrame,
                 hard_constraints: str,
                 num_ants: int = 10, 
                 evaporation_rate: float = 0.5,
                 alpha: int = 1,
                 beta: int = 2,
                 pheromone_init: float = 0.1,
                 pheromone_deposit: int = 1,
                 max_iterations: int = 100) -> None:
        self.ingredients_df = ingredients_df
        self.recipes_ingredients_df = recipes_ingredients_df
        self.allergens_df = allergens_df
        self.restriccions_df = restriccions_df
        self.hard_constraints = hard_constraints
        self.num_ants = num_ants
        self.evaporation_rate = evaporation_rate
        self.alpha = alpha
        self.beta = beta
        self.pheromone_init = pheromone_init
        self.pheromone_deposit = pheromone_deposit
        self.max_iterations = max_iterations
        self.num_ingredients = len(self.ingredients_df)
        self.pheromone_matrix = np.full((self.num_ingredients, self.num_ingredients), pheromone_init)
        self.best_solutions = []
        self.num_recipes = 7  # Number of recipes

    def _optimize(self) -> List[List[Ingredient]]:
        for _ in range(self.max_iterations):
            solutions = self._construct_solution()
            for solution in solutions:
                solution_fitness = self.evaluate_solution_fitness(solution)
                if len(self.best_solutions) < self.num_recipes:
                    self.best_solutions.append((solution, solution_fitness))
                else:
                    worst_fitness = max(self.best_solutions, key=lambda x: x[1])[1]
                    if solution_fitness < worst_fitness:
                        self.best_solutions.remove(max(self.best_solutions, key=lambda x: x[1]))
                        self.best_solutions.append((solution, solution_fitness))
            
           # Compare the generated recipe with existing ones and replace if necessary
            for i in range(len(solutions)):
                for j in range(len(self.best_solutions)):
                    generated_recipe = [ing.id for ing in solutions[i].recipe]
                    existing_recipe = [ing.id for ing in self.best_solutions[j][0].recipe]
                    max_length = max(len(generated_recipe), len(existing_recipe))
                    generated_recipe += [0] * (max_length - len(generated_recipe))
                    existing_recipe += [0] * (max_length - len(existing_recipe))
                    hamming_distance = sum(generated_recipe[k] != existing_recipe[k] for k in range(max_length)) / max_length
                    if hamming_distance < 0.5:  # Hamming distance threshold
                        self.best_solutions[j] = (solutions[i], self.evaluate_solution_fitness(solutions[i]))
            
            self.update_pheromone_matrix(solutions)
        return [solution[0].recipe for solution in self.best_solutions]
    
    def _construct_solution(self) -> List[Ant]:
        ants = [Ant(recipe=[]) for _ in range(self.num_ants)]
        for ant in ants:
            recipe_length = random.randint(5, 15)
            selected_ingredients = set()  # To avoid repeating ingredients
            while len(ant.recipe) < recipe_length:
                next_ingredient = self._select_next_ingredient(ant.recipe, selected_ingredients)
                if next_ingredient is not None:
                    ant.recipe.append(next_ingredient)
                    selected_ingredients.add(next_ingredient.id)
                else:
                    break  # There are no more valid ingridients
        return ants
    
    def _select_next_ingredient(self, current_recipe: List[Ingredient], selected_ingredients: set) -> Ingredient:
        probabilities = []
        available_ingredients = [i for i in range(1, self.num_ingredients + 1) if i not in selected_ingredients]
        
        for ingredient_id in available_ingredients:
            if self._check_ingredient(ingredient_id):
                pheromone = sum(self.pheromone_matrix[ingredient_id - 1][i.id - 1] for i in current_recipe) if current_recipe else 1.0
                desirability = self._calculate_desirability(ingredient_id)
                probability = (pheromone ** self.alpha) * (desirability ** self.beta)
                probabilities.append((ingredient_id, probability))
        
        if not probabilities:
            return None
        
        total_probability = sum(prob for _, prob in probabilities)
        if total_probability == 0:
            return None

        probabilities = [(ing, prob / total_probability) for ing, prob in probabilities]
        
        selected_ingredient_id = random.choices(
            [ing for ing, _ in probabilities], 
            [prob for _, prob in probabilities]
        )[0]
        
        return self._create_ingredient(selected_ingredient_id)

    def _create_ingredient(self, ingredient_id: int) -> Ingredient:
        ingredient_data = self.recipes_ingredients_df[self.recipes_ingredients_df['ID_INGREDIENTE'] == ingredient_id].sample()
        ingredient_row = ingredient_data.iloc[0]
        return Ingredient(
            id=ingredient_id,
            quantity=ingredient_row['Cantidad'],
            total_fat=ingredient_row['Grasa'],
            saturated_fats=ingredient_row['Grasas_saturadas'],
            trans_fat=ingredient_row['Grasas_trans'],
            cholesterol=ingredient_row['Colesterol'],
            sodium=ingredient_row['Sodio'],
            carbohydrates=ingredient_row['Carbohidratos'],
            protein=ingredient_row['Proteina'],
            calories=ingredient_row['Calorias']
        )

    def _calculate_desirability(self, ingredient_id: int) -> float:
        return 1.0

    def _check_ingredient(self, ingredient_id: int) -> bool:
        constraints_list = self.hard_constraints.split(',')
        vegan = int(constraints_list[0])
        vegetarian = int(constraints_list[1])
        user_allergens = [int(x) for x in constraints_list[2:]]
        
        ingredient = self.ingredients_df[self.ingredients_df['ID'] == ingredient_id]
        if ingredient.empty:
            raise ValueError(f'Ingredient with id: {ingredient_id} not found')
        
        ingredient_vegan = ingredient['Vegano'].iloc[0]
        ingredient_vegetarian = ingredient['Vegetariano'].iloc[0]
        
        if vegan and not ingredient_vegan:
            return False
        if vegetarian and not ingredient_vegetarian:
            return False
        
        ingredient_allergens = self.restriccions_df[self.restriccions_df['ID_INGREDIENTE'] == ingredient_id]['ID_ALERGENO'].tolist()
        if set(user_allergens).intersection(set(ingredient_allergens)):
            return False
        
        return True
    
    def evaluate_solution_fitness(self, solution: Ant) -> float:
        total_nutrition = self._calculate_total_nutrition(solution.recipe)
        fitness = 0

        target_ranges = {
            'Grasa': (13, 27),
            'Grasas_saturadas': (5, 7),
            'Grasas_trans': (0, 0.3),
            'Colesterol': (0.05, 0.1),
            'Sodio': (0.5, 0.8),
            'Carbohidratos': (75, 108),
            'Proteina': (15, 35),
            'Calorias': (600, 700)
        }

        for nutrient, (min_val, max_val) in target_ranges.items():
            if total_nutrition[nutrient] < min_val:
                fitness += (min_val - total_nutrition[nutrient]) ** 2
            elif total_nutrition[nutrient] > max_val:
                fitness += (total_nutrition[nutrient] - max_val) ** 2

        return fitness
    
    def _calculate_total_nutrition(self, recipe: List[Ingredient]) -> pd.Series:
        total_nutrition = pd.Series(
            data={
                'Cantidad': 0,
                'Grasa': 0,
                'Grasas_saturadas': 0,
                'Grasas_trans': 0,
                'Colesterol': 0,
                'Sodio': 0,
                'Carbohidratos': 0,
                'Proteina': 0,
                'Calorias': 0
            }
        )
        for ingredient in recipe:
            total_nutrition += pd.Series(
                data={
                    'Cantidad': ingredient.quantity,
                    'Grasa': ingredient.total_fat,
                    'Grasas_saturadas': ingredient.saturated_fats,
                    'Grasas_trans': ingredient.trans_fat,
                    'Colesterol': ingredient.cholesterol,
                    'Sodio': ingredient.sodium,
                    'Carbohidratos': ingredient.carbohydrates,
                    'Proteina': ingredient.protein,
                    'Calorias': ingredient.calories
                }
            )
        return total_nutrition
    
    def update_pheromone_matrix(self, solutions: List[Ant]) -> None:
        self.pheromone_matrix *= (1 - self.evaporation_rate)
        for ant in solutions:
            fitness = self.evaluate_solution_fitness(ant)
            if fitness == 0:
                fitness = 1  # avoid zero division
            pheromone_contribution = self.pheromone_deposit / fitness
            for i in range(len(ant.recipe) - 1):
                 self.pheromone_matrix[ant.recipe[i].id - 1][ant.recipe[i+1].id - 1] += pheromone_contribution

if __name__ == "__main__":
    # First number: Vegan (0 no, 1 yes)
    # Second number: Vegetarian (0 no, 1 yes)
    # Rest of numbers: Ids of allergens
    hard_constraints = '0,0,1'

    ant_colony = AntColony(
        ingredients_df=ingredients_df, 
        recipes_ingredients_df=recipes_ingredients_df, 
        allergens_df=allergens_df, 
        restriccions_df=restriccions_df, 
        hard_constraints=hard_constraints
    )
    
    best_recipes = ant_colony._optimize()
    
    if best_recipes:
        for i, best_recipe in enumerate(best_recipes):
            print(f"Receta {i+1}:")
            recipe_ingredients = pd.DataFrame([ingredient.__dict__ for ingredient in best_recipe])
            merged_df = recipe_ingredients.merge(ingredients_df, left_on='id', right_on='ID')
            for index, row in merged_df.iterrows():
                print(f'''ID: {row['id']},
                      Name: {row['Nombre']},
                      Quantity: {row['quantity']}, 
                      Total Fat: {row['total_fat']}, Saturated Fats: {row['saturated_fats']},
                      Trans Fat: {row['trans_fat']}, Cholesterol: {row['cholesterol']},
                      Sodium: {row['sodium']}, Carbohydrates: {row['carbohydrates']},
                      Protein: {row['protein']}, Calories: {row['calories']}''')
            print("\n")
    else:
        print("No valid recipes found.")

Receta 1:
ID: 41,
                      Name: Menta,
                      Quantity: 100.0, 
                      Total Fat: 0.7, Saturated Fats: 0.2,
                      Trans Fat: 0.0, Cholesterol: 0.0,
                      Sodium: 0.03, Carbohydrates: 8.4,
                      Protein: 3.3, Calories: 44.0
ID: 73,
                      Name: Panko,
                      Quantity: 25.0, 
                      Total Fat: 0.8, Saturated Fats: 0.0,
                      Trans Fat: 0.0, Cholesterol: 0.0,
                      Sodium: 0.04, Carbohydrates: 17.5,
                      Protein: 2.4, Calories: 87.5
ID: 78,
                      Name: Mix de especias mexicanas,
                      Quantity: 2.7, 
                      Total Fat: 0.2, Saturated Fats: 0.1,
                      Trans Fat: 0.0, Cholesterol: 0.0,
                      Sodium: 0.0, Carbohydrates: 1.7,
                      Protein: 0.2, Calories: 7.5
ID: 36,
                      Name: Chalota,
              

## Statistical Tests Hamming distance

In [13]:
import time
import statistics as stats
from scipy.stats import ttest_1samp, wilcoxon
import numpy as np

def run_experiment(ant_colony, num_runs):
    fitness_scores = []
    convergence_rates = []
    computational_times = []

    for _ in range(num_runs):
        start_time = time.time()
        best_recipes = ant_colony._optimize()
        end_time = time.time()

        # Evaluate fitness scores
        run_fitness_scores = [ant_colony.evaluate_solution_fitness(Ant(recipe=recipe)) for recipe in best_recipes]
        fitness_scores.extend(run_fitness_scores)

        # Calculate convergence rate
        unique_fitness_scores = set(run_fitness_scores)
        convergence_rate = len(unique_fitness_scores) / len(run_fitness_scores)
        convergence_rates.append(convergence_rate)

        # Record computational time
        computational_times.append(end_time - start_time)

    return {
        "fitness_scores": fitness_scores,
        "average_fitness_score": stats.mean(fitness_scores),
        "convergence_rate": stats.mean(convergence_rates),
        "computational_time": stats.mean(computational_times),
        "fitness_score_std_dev": stats.stdev(fitness_scores)
    }

# Run the experiment
num_runs = 5
experiment_results = run_experiment(ant_colony, num_runs)

# Output results
print("Average Fitness Score:", experiment_results["average_fitness_score"])
print("Convergence Rate:", experiment_results["convergence_rate"])
print("Computational Time:", experiment_results["computational_time"])
print("Fitness Score Standard Deviation:", experiment_results["fitness_score_std_dev"])

# Additional tests
fitness_scores_array = np.array(experiment_results["fitness_scores"])

# Mean and Standard Deviation
print("Mean of Fitness Scores:", np.mean(fitness_scores_array))
print("Standard Deviation of Fitness Scores:", np.std(fitness_scores_array))

# T-tests or ANOVA
# Example t-test comparing to a hypothetical mean value (e.g., 100)
t_statistic, p_value = ttest_1samp(fitness_scores_array, 100)
print("T-statistic:", t_statistic)
print("P-value:", p_value)

# Wilcoxon Signed-Rank Test
# Example Wilcoxon signed-rank test comparing to a hypothetical median value (e.g., 100)
w_statistic, p_value = wilcoxon(fitness_scores_array - 100)
print("Wilcoxon Signed-Rank Test Statistic:", w_statistic)
print("P-value:", p_value)


Average Fitness Score: 1.2680685714285707
Convergence Rate: 0.5428571428571428
Computational Time: 198.95178432464598
Fitness Score Standard Deviation: 0.8180777874604186
Mean of Fitness Scores: 1.2680685714285707
Standard Deviation of Fitness Scores: 0.8063062703363237
T-statistic: -713.9980970320748
P-value: 1.3866624615786085e-72
Wilcoxon Signed-Rank Test Statistic: 0.0
P-value: 5.820766091346741e-11


# Conclusions

**Conclusions:**<br>
Observing the results of the 3 statistical tests we conducted, we can affirm that the best version overall is the one that does not use metrics to compare the similarity between recipes, that is, the second version. This is because it has the lowest average fitness score, indicating that the solutions are the closest to the desired nutritional goals. Additionally, the standard deviation is also low, suggesting that the solutions are consistent. The moderate convergence rate suggests a good balance between exploration and exploitation, with no clear signs of premature convergence.
<br><br>
**Key Points:** <br>
* Second Version: Best average fitness, high consistency, good exploration without premature convergence.
* Third Version (Jaccard Distance): Greater variability in solutions, signs of premature convergence, less optimal solutions.
* Third Version (Hamming Distance): Better than the second in terms of fitness and consistency, but not as good as the first. Moderate convergence rate indicates good diversity.<br><br>
The statistical results of the **second version** of the algorithm suggest that it excels in balancing exploration and exploitation, resulting in optimal and consistent solutions, making it **the best overall option**.